In [ ]:
import os
import numpy as np

from keras.models import load_model  # TensorFlow is required for Keras to work
from PIL import Image, ImageOps  # Install pillow instead of PIL
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
if 'google.colab' in str(get_ipython()):
    from google.colab import drive
    drive.mount('/content/drive')

In [ ]:
PROJECT_DIR = '/content/drive/MyDrive/Colab Notebooks/teachable-gradio'
PROJECT_NAME = 'sample_project'
TEACHABLE_MODEL_DIR = f'{PROJECT_DIR}/{PROJECT_NAME}/converted_keras'
TEACHABLE_TRAIN_DIR = f'{PROJECT_DIR}/{PROJECT_NAME}/train'
TEACHABLE_TEST_DIR = f'{PROJECT_DIR}/{PROJECT_NAME}/test'
SHAPE = (224, 224)

In [ ]:
# load the model and label
model = load_model(f"{TEACHABLE_MODEL_DIR}/keras_model.h5", compile=False)
class_names = open(f"{TEACHABLE_MODEL_DIR}/labels.txt", "r").readlines()
class_names = [cn.replace('\n', '') for cn in class_names]


def pre_process_image(image):
    # resizing the image to be at least 224x224 and then cropping from the center
    image = ImageOps.fit(image, SHAPE, Image.LANCZOS)

    # turn the image into a numpy array
    image_array = np.asarray(image)

    # normalize the image
    normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

    # reshape the image to batch format
    reshaped_image_array = normalized_image_array.reshape((-1, SHAPE[0], SHAPE[1], 3))

    return reshaped_image_array


def classify_image(image):
    image = pre_process_image(image)
    prediction = model.predict(image)
    confidences = {class_names[i]: float(prediction[0][i]) for i in range(len(class_names))}
    return confidences

In [ ]:
def evaluate_model_on_dataset(dataset_dir):
    true_labels = []
    predicted_labels = []

    for class_name in os.listdir(dataset_dir):
        class_dir = os.path.join(dataset_dir, class_name)
        for image_name in os.listdir(class_dir):
            image_path = os.path.join(class_dir, image_name)
            image = Image.open(image_path)
            true_class_idx = class_names.index(class_name)

            image = pre_process_image(image)
            prediction = model.predict(image, verbose=0)
            predicted_class_idx = np.argmax(prediction)

            true_labels.append(true_class_idx)
            predicted_labels.append(predicted_class_idx)

    cm = confusion_matrix(true_labels, predicted_labels)
    report = classification_report(true_labels, predicted_labels, target_names=class_names)

    return cm, report

# Evaluate on the training set
train_cm, train_report = evaluate_model_on_dataset(TEACHABLE_TRAIN_DIR)
print("Training Set Confusion Matrix:")
print(train_cm)
print("Training Set Classification Report:")
print(train_report)

# Evaluate on the validation/test set
val_cm, val_report = evaluate_model_on_dataset(TEACHABLE_TEST_DIR)
print("Validation Set Confusion Matrix:")
print(val_cm)
print("Validation Set Classification Report:")
print(val_report)